# Mangrove Kandel Backtester - One sample


## Engine


### 0. Imports


In [1]:
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from src.kandel_backtester import KandelBacktester
from src.kandel_v2 import KandelConfig

pd.options.plotting.backend = "plotly"

In [2]:
with open("config.json") as f:
    config = json.load(f)


window = config["window"]
initial_quote = config["initial_quote"]
initial_base = config["initial_base"]
vol_mult = config["vol_mult"]
n_points = config["n_points"]
step_size = config["step_size"]
vol_threshold_window = config["vol_threshold_window"]

del config

### 1. Preprocessing


#### Load data


In [3]:
prices_df = pd.read_csv("data/binance/ETHUSDC-2024-01>08_1s.csv", header=0, index_col=0)
prices_df.index = pd.to_datetime(prices_df.index, unit="s", utc=True)
prices_df["log_return"] = np.log(prices_df["price"] / prices_df["price"].shift(1))

In [4]:
prices_df["exit_vol"] = prices_df["log_return"].rolling(
    vol_threshold_window * 3600
).std() * np.sqrt(vol_threshold_window * 3600)
prices_df["exit_vol"] = prices_df["exit_vol"].fillna(0)

In [5]:
prices_df["window_vol"] = prices_df["log_return"].rolling(
    window * 3600
).std() * np.sqrt(window * 3600)
prices_df["window_vol"] = prices_df["window_vol"].fillna(0)

In [6]:
prices_df = prices_df.drop(columns=["log_return"])

#### Create sample


In [7]:
# Remove days used for volatility calculation
prices_df_trimmed = prices_df.iloc[(vol_threshold_window + window) * 3600 :10*24*3600]
initial_spot = prices_df_trimmed["price"].iloc[0]

del prices_df

### 2. Run backtester


In [8]:
kandel_backtester = KandelBacktester(
    prices=prices_df_trimmed["price"],
    config=KandelConfig(
        window=window * 3600,
        initial_quote=initial_quote,
        initial_base=initial_base / initial_spot,
        vol_mult=vol_mult,
        n_points=n_points,
        step_size=step_size,
        vol_threshold_window=vol_threshold_window,
    ),
    window_vol=prices_df_trimmed["window_vol"],
    exit_vol=prices_df_trimmed["exit_vol"],
)

/Users/nicolasdebengy/Documents/LP/mangrove-kandel-backtester/src/kandel_backtester.py:61: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.kandel = Kandel(config, prices[0], window_vol[0])


In [9]:
res, order_book_history, transaction_history = kandel_backtester.run()

100%|██████████| 734400/734400 [00:14<00:00, 50301.20it/s]


In [19]:
res.index = prices_df_trimmed.index
res["price"] = prices_df_trimmed["price"]
res["window_vol"] = prices_df_trimmed["window_vol"]
res["exit_vol"] = prices_df_trimmed["exit_vol"]
res["mtm_quote"] = res["quote"] + res["base"] * res["price"]
res["mtm_base"] = res["base"] + res["quote"] / res["price"]
res["returns_quote"] = res["mtm_quote"] / (initial_quote + initial_base) - 1
res["returns_base"] = (
    res["mtm_base"] / ((initial_quote + initial_base) / initial_spot) - 1
)

res_1h = res.resample("1H").last()

In [22]:
order_book_history_df = pd.DataFrame.from_records(
    [ob.to_dict() for ob in order_book_history]
)
order_book_history_df.index = res.index
order_book_history_1h = order_book_history_df.resample("1h").last()

## Analysis

In [26]:
window_hours = window // 3600 - 1
days = len(res_1h) // 24

# PLOT
fig = make_subplots(
    rows=4,
    cols=1,
    shared_xaxes=True,
    specs=[[{"secondary_y": True}], [{}], [{"secondary_y": True}], [{"secondary_y": True}]],
    vertical_spacing=0.05,
    subplot_titles=("MTM", "Returns", "Price", "Balance"),
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index,
        y=res_1h.mtm_quote,
        mode="lines",
        name="MTM in USDC",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index,
        y=res_1h.mtm_base,
        mode="lines",
        name="MTM in ETH",
    ),
    row=1,
    col=1,
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index,
        y=res_1h.returns_quote,
        mode="lines",
        name="Over holding USDC",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index,
        y=res_1h.returns_base,
        mode="lines",
        name="Over holding ETH",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index,
        y=res_1h.price,
        mode="lines",
        name="Price",
    ),
    row=3,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index,
        y=res_1h.window_vol,
        mode="lines",
        name="HV"
    ),
    secondary_y=True,
    row=3,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index,
        y=res_1h.quote,
        name="Quote size",
        legendgroup="Quote",
        stackgroup="1",
    ),
    row=4,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index,
        y=-res_1h.base,
        name="Base size",
        legendgroup="Base",
        stackgroup="1",
    ),
    row=4,
    col=1,
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index,
        y=-res_1h.quote,
        name="Quote size",
        legendgroup="Quote",
        marker=dict(color="rgba(0,0,0,0)")
    ),
    row=4,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index,
        y=res_1h.base,
        name="Base size",
        legendgroup="Base",
        marker=dict(color="rgba(0,0,0,0)"),
    ),
    row=4,
    col=1,
    secondary_y=True,
)

fig.update_layout(
    height=1200,
    width=1600,
    title_text=f"MTM, Returns and Price.<br><sup>window = {window} | quote = {initial_quote} | base = {initial_base} | vol_mult = {vol_mult} | n_points = {n_points} | step_size = {step_size} | days = {days}</sup>",
)

fig.update_yaxes(row=2, col=1, tickformat=".2%")

fig.show()

In [24]:
fig = make_subplots(
    rows=1,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=(""),
    specs=[[{"secondary_y": True}]],
)

for i, ob in enumerate(order_book_history_1h.iterrows()):
    bids = ob[1].bids
    asks = ob[1].asks

    fig.add_trace(
        go.Scatter(
            x=[i for _ in range(len(bids))],
            y=tuple(bids),
            mode="markers",
            name="Bids",
            showlegend=False,
            marker=dict(
                color="green",
                symbol="line-ew-open",
                size=8,
                line=dict(width=5, color="black"),
            ),
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=[i for _ in range(len(asks))],
            y=tuple(asks),
            mode="markers",
            name="Asks",
            showlegend=False,
            marker=dict(
                color="red",
                symbol="line-ew-open",
                size=8,
                line=dict(width=5, color="black"),
            ),
        ),
    )

fig.add_trace(
    go.Scatter(
        x=[i for i in range(len(order_book_history_1h))],
        y=res_1h.price,
        mode="lines",
        name="Price",
        marker=dict(color="blue"),
        hovertext=[
            f"Price: {res_1h.price.iloc[i]:.2f}<br>Time: {res_1h.index[i].strftime('%b %d %H:%M')}"
            for i in range(len(res_1h))
        ]
    )
)

fig.update_layout(
    height=1000,
    width=1600,
    title_text=f"Order book<br><sup>window = {window} | quote = {initial_quote} | base = {initial_base} | vol_mult = {vol_mult} | n_points = {n_points} | step_size = {step_size} | days = {days}</sup>",
)

fig.update_xaxes(
    tickvals=np.arange(0, len(res_1h[window_hours:]), 24),
    ticktext=[res_1h.index[i].strftime("%b %d, %Hh") for i in np.arange(0, len(res_1h), 24)],
)

fig.show()